### 행정안전부 도로명주소 API 활용

In [1]:
import requests, json
from urllib.parse import quote      # 한글 인코딩

- API Key 가져오기

In [3]:
with open('data/roadapikey.txt') as f:
    road_key = f.read()

- URL 만들기

In [5]:
# 한글은 반드시 인코딩해서 보내야 함
quote('경기도청')

'%EA%B2%BD%EA%B8%B0%EB%8F%84%EC%B2%AD'

In [6]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote("경기도청")}&resultType=json'
url = f'{base_url}?{params1}&{params2}'

- Open API를 호출하여 결과 받기

In [7]:
result = requests.get(url)
result.status_code

200

In [10]:
result.text[:200]

'({"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"68","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"","engAddr":"30 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeongg'

- JSON data를 python에서 읽을 수 있도록 변환

In [11]:
# 시작과 끝에 ( )가 있어, 이것을 없애주어야 함
res = json.loads(result.text[1:-1])

- 딕셔너리 데이터에서 필요한 정보 추출하기

In [16]:
res.keys()

dict_keys(['results'])

In [18]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [25]:
type(res['results']['juso'])

list

In [26]:
res['results']['juso'][0]

{'detBdNmList': '',
 'engAddr': '30 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeonggi-do',
 'rn': '도청로',
 'emdNm': '이의동',
 'zipNo': '16508',
 'roadAddrPart2': ' (이의동)',
 'emdNo': '01',
 'sggNm': '수원시 영통구',
 'jibunAddr': '경기도 수원시 영통구 이의동 0 경기도청',
 'siNm': '경기도',
 'roadAddrPart1': '경기도 수원시 영통구 도청로 30',
 'bdNm': '경기도청',
 'admCd': '4111710300',
 'udrtYn': '0',
 'lnbrMnnm': '0',
 'roadAddr': '경기도 수원시 영통구 도청로 30 (이의동)',
 'lnbrSlno': '0',
 'buldMnnm': '30',
 'bdKdcd': '0',
 'liNm': '',
 'rnMgtSn': '411173177029',
 'mtYn': '0',
 'bdMgtSn': '4111710300100000000000001',
 'buldSlno': '0'}

In [27]:
# 원하는 결과
res['results']['juso'][0]['roadAddr']

'경기도 수원시 영통구 도청로 30 (이의동)'

#### 경기도청, 장안구청, 권선구청, 팔달구청, 영통구청

In [28]:
gov_list = '경기도청, 장안구청, 권선구청, 팔달구청, 영통구청'.split(', ')
gov_list

['경기도청', '장안구청', '권선구청', '팔달구청', '영통구청']

In [30]:
road_addr_list = []
for gov in gov_list:
    base_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
    params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
    params2 = f'keyword={quote(gov)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    if result.status_code == 200:
        res = json.loads(result.text[1:-1])
        road_addr_list.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)

In [40]:
import pandas as pd

df = pd.DataFrame({
    '관공서명': gov_list, '도로명 주소': road_addr_list
})
df

,관공서명,도로명 주소
0,경기도청,경기도 수원시 영통구 도청로 30 (이의동)
1,장안구청,경기도 수원시 장안구 송원로 101 (조원동)
2,권선구청,경기도 수원시 권선구 호매실로 12 (탑동)
3,팔달구청,경기도 수원시 팔달구 창룡대로 23 (매향동)
4,영통구청,경기도 수원시 영통구 효원로 407 (매탄동)


In [41]:
df.to_csv('data/수원소재관공서.csv', index=False)
pd.read_csv('data/수원소재관공서.csv')

,관공서명,도로명 주소
0,경기도청,경기도 수원시 영통구 도청로 30 (이의동)
1,장안구청,경기도 수원시 장안구 송원로 101 (조원동)
2,권선구청,경기도 수원시 권선구 호매실로 12 (탑동)
3,팔달구청,경기도 수원시 팔달구 창룡대로 23 (매향동)
4,영통구청,경기도 수원시 영통구 효원로 407 (매탄동)
